In [8]:
import os
import json
import datetime
import time

from dateutil import parser
import glob
import json
from dataclasses import dataclass, field
from dacite import from_dict, Config
from typing import Optional
import pandas as pd

CREDENTIALS_FILE = "../creds.txt"

# Load Twitter API OAuth and other details

from TwitterAPI import TwitterAPI, TwitterOAuth

o = TwitterOAuth.read_file(CREDENTIALS_FILE)
api = TwitterAPI(o.consumer_key, o.consumer_secret, o.access_token_key, o.access_token_secret, auth_type="oAuth2", api_version="2")


directory = "../datasets/users/ia"
filename = "users-ia.json"
filepath = os.path.join(directory, filename)


In [9]:
# user_ids = set()
# with open(filepath, "r") as f:
#     for line in f:
#         user_ids.add(json.loads(line.strip())["id"])
df = pd.read_pickle("users_ia_hk.pkl")
users = set(df.loc[df["deleted"] == False]["id"])

In [10]:
user_id = users.pop()
# def batch(l, n):
#     for i in range(0, len(l), n):
#         yield l[i:i+n]

# Example
#     o = TwitterOAuth.read_file()
#     api = TwitterAPI(o.consumer_key, o.consumer_secret, o.access_token_key, o.access_token_secret, api_version='2')

#     # Get tweets - default setting
#     tweets = api.request(f'users/:{USER_ID}/tweets')
#     for t in tweets:
#         print(t)

#     # Get tweets with customization - (5 tweets only with created_at timestamp)
#     print()
#     params = {'max_results': 5, 'tweet.fields': 'created_at'}
#     tweets = api.request(f'users/:{USER_ID}/tweets', params)
#     for t in tweets:
#         pprint(t)
        
#     # Get next 5 tweets
#     print()
#     next_token = tweets.json()['meta']['next_token']
#     params = {'max_results': 5, 'tweet.fields': 'created_at', 'pagination_token': next_token}
#     tweets = api.request(f'users/:{USER_ID}/tweets', params)
#     for t in tweets:
#         pprint(t)    

In [11]:
users_queried = {}

with open("user_timelines.json", "r") as f:
    for line in f:
        datum = json.loads(line.strip())
        users_queried[datum["id"]] = datum

In [12]:
n_requests = 0
n_users = 0
for user_id in users:
    n_users += 1
    if user_id in users_queried:
        continue
    print(f"{n_users}/{len(users)}:{user_id}")
    params = {
        "tweet.fields": "id,text,author_id,created_at,withheld",
        "expansions": "geo.place_id", #,referenced_tweets.id",
        "max_results": 500,
        "start_time": datetime.date(2019,1,1).strftime('%Y-%m-%dT%H:%M:%SZ'),
        "end_time": datetime.date(2021,1,1).strftime('%Y-%m-%dT%H:%M:%SZ'),
        "query": f"from:{user_id}"
    }
    with open("timelines.json", "a+") as f:
        count = 0
        n_tweets = 0
        earliest = None
        last = None
        while True:
            count += 1
            n_requests += 1
            
            if n_requests >= 300:
                print("Sleeping 15 minutes for ")
                time.sleep(60*15)
                n_requests = 0
            time.sleep(1)
            count_tweets = api.request(f"tweets/counts/all", 
                                       {"query": params["query"], "start_time": params["start_time"], "end_time": params["end_time"], "granularity": "day"})
            count_data = count_tweets.json()
            n_tweets = count_data["meta"]["total_tweet_count"]
            if n_tweets > 10000:
                with open("user_timelines.json", "a+") as f:
                    json.dump({"id": user_id, "num_tweets": n_tweets, "earliest": None, "last": None }, f)
                    f.write("\n")
                break
            print(f"{n_tweets} tweets to pull")
            time.sleep(1)
            tweets = api.request(f"tweets/search/all", params)
            
            for t in tweets:
                if earliest is None:
                    earliest = t["created_at"]
                json.dump(t, f)
                n_tweets += 1
                last = t["created_at"]
                f.write("\n")
            if count % 100 == 0:
                print(t["created_at"])
            meta = tweets.json()["meta"]
            if "next_token" in meta:
                next_token = meta["next_token"]
                params["next_token"] = next_token
            else:
                break
        with open("user_timelines.json", "a+") as f:
            json.dump({"id": user_id, "num_tweets": n_tweets, "earliest": earliest, "last": last }, f)
            f.write("\n")

1541/17471:398658332
1542/17471:890956512553689092
1543/17471:1100920451184812038
1544/17471:1053577105793966086
1545/17471:2373356323
1546/17471:1056561571789365255
1547/17471:128617253
1548/17471:1120196355517861889
1549/17471:2607057714
1550/17471:256379703
1551/17471:1219627831
1552/17471:144771897
1553/17471:874071823662497792
1554/17471:348392252
1555/17471:2242087742
1556/17471:635898686
1557/17471:770489938680934400
1558/17471:701377373082677248
1559/17471:978595638064525312
1560/17471:754684248292851712
1561/17471:3224997012
1562/17471:291572554
1563/17471:2170456915
1564/17471:1034368643407048704
1565/17471:4694477658
1566/17471:1229130588
1567/17471:1103934782277214209
1568/17471:3565194079
2019-10-04T09:31:16.000Z
2019-03-04T08:40:40.000Z
Sleeping 15 minutes for 
1569/17471:1118304537444409344
1570/17471:55151456
1571/17471:1028344265586302976
1572/17471:1038321842879049728
1573/17471:2386004840
1574/17471:256052073
1575/17471:134122350
1576/17471:1144503631716032512
1577/1

KeyboardInterrupt: 

In [32]:
all_tweets

[]